## GOAL 3

__- Find correlations between the features (Education, cost, ...) and the demographics__

In [90]:
import pandas as pd
import numpy as np
import pycountry
import statistics as sts

In [3]:
features_sur = pd.read_csv('Features.csv')
personalty_sur = pd.read_csv('Personality_.csv')
print('# user complete Feature surevey:',len(features_sur))
print('# user comletd Personality surevy:',len(personalty_sur))

# user complete Feature surevey: 226
# user comletd Personality surevy: 241


In [4]:
features = features_sur[['user_id','features']]
features.head(3)
features.tail(3)

,user_id,features
223,butt,"Education Quality, Political Insecurity, Socia..."
224,Insomnia1305,"Work Opportunities, Income Difference, Cultura..."
225,Sai,"Education Quality, Political Insecurity, Socia..."


In [130]:
demographic = personalty_sur[['user_id','age','gender','country','imaginative','organized','enthusiastic','kind','calm']]
demographic.tail(3)

,user_id,age,gender,country,imaginative,organized,enthusiastic,kind,calm
238,butt,b25_35,Male,IN,Neutral,Disagree,Disagree,Agree,Agree
239,Insomnia1305,b25_35,Male,GB,Agree,Agree,Strongly_Agree,Neutral,Disagree
240,Sai,under_18,Male,IN,Agree,Disagree,Agree,Agree,Strongly_Disagree


In [186]:
# Create a joint between two features and dimographic dataframe
feature_dimographic = pd.merge(demographic, features, on='user_id', how='inner')

In [195]:
features_dimo = feature_dimographic
feature_dimographic.head()

,user_id,age,gender,country,imaginative,organized,enthusiastic,kind,calm,features
0,Eggbjj,b35_45,Male,IL,Agree,Strongly_Agree,Agree,Agree,Agree,"Political Insecurity, Health Care, Wars & Dict..."
1,rihi,b18_24,Male,MA,Strongly_Agree,Disagree,Agree,Neutral,Neutral,"Education Quality, Political Insecurity, Socia..."
2,Test,b18_24,Male,MA,Agree,Agree,Agree,Agree,Agree,"Education Quality, Work Opportunities, Income ..."
3,hamza0095,b18_24,Male,MA,Agree,Agree,Neutral,Neutral,Agree,"Education Quality, Work Opportunities, Family ..."
4,abdelbari,b18_24,Male,MA,Agree,Agree,Neutral,Neutral,Neutral,"Education Quality, Work Opportunities, Health ..."


In [133]:
feature_dimographic.features = feature_dimographic.features.str.split(',')

In [134]:
# Prepocessing the Data in order to match each feature with user
lst_col = 'features'
Data_features_dimographics = pd.DataFrame({
      col:np.repeat(feature_dimographic[col].values, feature_dimographic[lst_col].str.len())
      for col in feature_dimographic.columns.drop(lst_col)}
    ).assign(**{lst_col:np.concatenate(feature_dimographic[lst_col].values)})[feature_dimographic.columns]

In [178]:
Data_features_dimographics.features = Data_features_dimographics.features.str.strip()
Data_features_dimographics.to_csv('./csv_result/Data_features_dimographics.csv')
Data_features_dimographics.head()

,user_id,age,gender,country,imaginative,organized,enthusiastic,kind,calm,features
0,Eggbjj,b35_45,Male,IL,Agree,Strongly_Agree,Agree,Agree,Agree,Political Insecurity
1,Eggbjj,b35_45,Male,IL,Agree,Strongly_Agree,Agree,Agree,Agree,Health Care
2,Eggbjj,b35_45,Male,IL,Agree,Strongly_Agree,Agree,Agree,Agree,Wars & Dictatorship
3,rihi,b18_24,Male,MA,Strongly_Agree,Disagree,Agree,Neutral,Neutral,Education Quality
4,rihi,b18_24,Male,MA,Strongly_Agree,Disagree,Agree,Neutral,Neutral,Political Insecurity


In [136]:
def perce(dim):
    count = Data_features_dimographics[str(dim)].value_counts()
    count_per = Data_features_dimographics[str(dim)].value_counts(normalize = True).mul(100).round(1).astype('str') + '%'
    count_pers = {}
    for typ,value,perse in zip(count.keys(),count, count_per):
        count_pers[typ] = [str(value)+' (' +str(perse)+')']
    count_percentage = pd.DataFrame.from_dict(count_pers)
    return count_percentage

### Overall Feature Preference

In [137]:
t = perce('features').T
t.columns = ['# Respondent']
t

,# Respondent
Work Opportunities,161 (19.4%)
Education Quality,105 (12.6%)
Working Atmosphere,100 (12.0%)
Health Care,97 (11.7%)
Income Difference,84 (10.1%)
Political Insecurity,59 (7.1%)
Crime Rate,58 (7.0%)
Social Conflict,49 (5.9%)
Cultural & Linguistic Similarities,47 (5.6%)
Wars & Dictatorship,37 (4.4%)


### Features by gender

In [138]:
t = Data_features_dimographics[['gender','features']].groupby(['gender','features']).size()

In [139]:
t

gender          features                          
Female          Crime Rate                             12
                Cultural & Linguistic Similarities      9
                Education Quality                      22
                Family Member Abroad                   12
                Health Care                            24
                Income Difference                      11
                Political Insecurity                   14
                Shorter Distance                        4
                Social Conflict                        12
                Wars & Dictatorship                     9
                Work Opportunities                     40
                Working Atmosphere                     31
Male            Crime Rate                             44
                Cultural & Linguistic Similarities     36
                Education Quality                      81
                Family Member Abroad                    7
                Healt

### Respondent Origin Country and Features

In [140]:
t = Data_features_dimographics[['country','features']].groupby(['country','features']).size()

In [141]:
t.to_csv('Origin_country_Features.csv', header=True)
a = pd.read_csv('./csv_result/Origin_country_features.csv')
a['country'] = a['country'].apply(lambda x: pycountry.countries.get(alpha_3=x).name if len(x) == 3 else pycountry.countries.get(alpha_2=x).name)

In [142]:
a

,country,features,number
0,United Arab Emirates,Crime Rate,4
1,United Arab Emirates,Education Quality,2
2,United Arab Emirates,Family Member Abroad,2
3,United Arab Emirates,Health Care,2
4,United Arab Emirates,Income Difference,3
5,United Arab Emirates,Political Insecurity,2
6,United Arab Emirates,Wars & Dictatorship,1
7,United Arab Emirates,Work Opportunities,6
8,Afghanistan,Family Member Abroad,1
9,Afghanistan,Income Difference,1


### Features and Age

In [153]:
feature_age = Data_features_dimographics[['age','features']].groupby(['age','features']).size()

In [154]:
feature_age

age       features                          
b18_24    Crime Rate                            23
          Cultural & Linguistic Similarities    17
          Education Quality                     55
          Family Member Abroad                  11
          Health Care                           38
          Income Difference                     38
          Political Insecurity                  15
          Shorter Distance                       5
          Social Conflict                       19
          Wars & Dictatorship                   10
          Work Opportunities                    71
          Working Atmosphere                    45
b25_35    Crime Rate                            20
          Cultural & Linguistic Similarities    17
          Education Quality                     30
          Family Member Abroad                   4
          Health Care                           38
          Income Difference                     33
          Political Insecurity       

# AB Group Split

In [196]:
def perce(dim):
    count = features_dimo[str(dim)].value_counts()
    count_per = features_dimo[str(dim)].value_counts(normalize = True).mul(100).round(1).astype('str') + '%'
    count_pers = {}
    for typ,value,perse in zip(count.keys(),count, count_per):
        count_pers[typ] = [str(value)+' (' +str(perse)+')']
    count_percentage = pd.DataFrame.from_dict(count_pers)
    return count_percentage

In [191]:
personality_traits.head(3)

,user_id,age,gender,imaginative,organized,enthusiastic,kind,calm
0,Eggbjj,b35_45,Male,Agree,Strongly_Agree,Agree,Agree,Agree
1,rihi,b18_24,Male,Strongly_Agree,Disagree,Agree,Neutral,Neutral
2,Test,b18_24,Male,Agree,Agree,Agree,Agree,Agree


### Imaginative

In [198]:
# Imaginative
print(perce('imaginative'))
print('the median:', sts.median([120,79,30,7,5]))

         Agree Strongly_Agree     Neutral  Disagree Strongly_Disagree
0  114 (50.4%)     76 (33.6%)  26 (11.5%)  7 (3.1%)          3 (1.3%)
the median: 30


### organized

In [199]:
print(perce('organized').head())
print('the median:', sts.median([107,60,41,27,6]))

         Agree     Neutral Strongly_Agree    Disagree Strongly_Disagree
0  100 (44.2%)  56 (24.8%)     39 (17.3%)  25 (11.1%)          6 (2.7%)
the median: 41


### enthusiastic

In [200]:
print(perce('enthusiastic').head())
print('the median:', sts.median([70,66,61,27,17]))

      Neutral       Agree    Disagree Strongly_Agree Strongly_Disagree
0  66 (29.2%)  64 (28.3%)  53 (23.5%)     26 (11.5%)         17 (7.5%)
the median: 61


### kind

In [201]:
print(perce('kind').head())
print('the median:', sts.median([119,62,38,16,6]))

         Agree Strongly_Agree     Neutral   Disagree Strongly_Disagree
0  111 (49.1%)     59 (26.1%)  34 (15.0%)  16 (7.1%)          6 (2.7%)
the median: 38


### calm

In [202]:
print(perce('calm').head())
print('the median:', sts.median([89,63,60,22,7]))

        Agree Strongly_Agree     Neutral   Disagree Strongly_Disagree
0  82 (36.3%)     60 (26.5%)  57 (25.2%)  21 (9.3%)          6 (2.7%)
the median: 60


Based on the median of each question, we can devied the responses into two group <br>
__Group A__ : **_Agree, Strongly Agree_**<br>
__Group B__ : **_Neutral, Disagree, Strongly Disagree_**<br>

### Features and Personality

In [152]:
Data_features_dimographics.head()

,user_id,age,gender,country,imaginative,organized,enthusiastic,kind,calm,features
0,Eggbjj,b35_45,Male,IL,Agree,Strongly_Agree,Agree,Agree,Agree,Political Insecurity
1,Eggbjj,b35_45,Male,IL,Agree,Strongly_Agree,Agree,Agree,Agree,Health Care
2,Eggbjj,b35_45,Male,IL,Agree,Strongly_Agree,Agree,Agree,Agree,Wars & Dictatorship
3,rihi,b18_24,Male,MA,Strongly_Agree,Disagree,Agree,Neutral,Neutral,Education Quality
4,rihi,b18_24,Male,MA,Strongly_Agree,Disagree,Agree,Neutral,Neutral,Political Insecurity


#### Imaginative/ features


In [255]:
t = Data_features_dimographics[['gender','age','imaginative','features']].groupby(['gender','age','imaginative','features']).size()
t.to_frame().to_csv('gender_imaginative_features.csv')
features_imaginative_data = pd.read_csv('gender_imaginative_features.csv')

In [256]:
features_imaginative_data.head()

,gender,age,imaginative,features,0
0,Female,b18_24,Agree,Crime Rate,2
1,Female,b18_24,Agree,Cultural & Linguistic Similarities,3
2,Female,b18_24,Agree,Education Quality,10
3,Female,b18_24,Agree,Family Member Abroad,6
4,Female,b18_24,Agree,Health Care,4


In [257]:
gender_imaginative = Data_features_dimographics[['user_id','gender','imaginative']].groupby(['user_id','gender','imaginative']).size()
age_imaginative = Data_features_dimographics[['user_id','age','imaginative']].groupby(['user_id','age','imaginative']).size()
gender_imaginative.to_frame().to_csv('gender_imaginative.csv')
age_imaginative.to_frame().to_csv('age_imaginative.csv')
gender_imaginative = pd.read_csv('gender_imaginative.csv')
gender_imaginative.head()

,user_id,gender,imaginative,0
0,1234567890,Female,Agree,3
1,1278,Male,Agree,3
2,Abc,Male,Disagree,3
3,Abdo,Male,Strongly_Agree,3
4,Abdo_mahdi,Male,Neutral,3


In [258]:
gender_imaginative.groupby(['imaginative','gender']).size()

imaginative        gender        
Agree              Female            30
                   Male              83
                   refuse_to_disc     1
Disagree           Male               6
                   refuse_to_disc     1
Neutral            Female             7
                   Male              18
                   refuse_to_disc     1
Strongly_Agree     Female            22
                   Male              53
                   refuse_to_disc     1
Strongly_Disagree  Male               2
                   refuse_to_disc     1
dtype: int64

In [259]:
age_imaginative.groupby(['imaginative','age']).size()

imaginative        age     
Agree              b18_24      49
                   b25_35      41
                   b35_45      11
                   b45_55       6
                   under_18     7
Disagree           b18_24       3
                   b25_35       2
                   b35_45       1
                   under_18     1
Neutral            b18_24      10
                   b25_35      12
                   b35_45       3
                   bover_55     1
Strongly_Agree     b18_24      34
                   b25_35      27
                   b35_45       7
                   b45_55       3
                   under_18     5
Strongly_Disagree  b25_35       1
                   b35_45       1
                   bover_55     1
dtype: int64

In [260]:
imaginative_features = Data_features_dimographics.groupby(['imaginative','features']).size()
imaginative_features.to_frame().to_csv('imaginative_features.csv')
imaginative_features = pd.read_csv('imaginative_features.csv')
imaginative_features

,imaginative,features,0
0,Agree,Crime Rate,27
1,Agree,Cultural & Linguistic Similarities,25
2,Agree,Education Quality,54
3,Agree,Family Member Abroad,10
4,Agree,Health Care,46
5,Agree,Income Difference,37
6,Agree,Political Insecurity,33
7,Agree,Shorter Distance,7
8,Agree,Social Conflict,26
9,Agree,Wars & Dictatorship,17


#### Organized/features

In [265]:
or_f = Data_features_dimographics[['gender','age','organized','features']].groupby(['gender','age','organized','features']).size()
or_f.to_frame().to_csv('gender_organized_features.csv')
features_organized_data = pd.read_csv('gender_organized_features.csv')

In [266]:
features_organized_data.head()

,gender,age,organized,features,0
0,Female,b18_24,Agree,Cultural & Linguistic Similarities,2
1,Female,b18_24,Agree,Education Quality,7
2,Female,b18_24,Agree,Family Member Abroad,3
3,Female,b18_24,Agree,Health Care,2
4,Female,b18_24,Agree,Income Difference,1


In [267]:
gender_organized = Data_features_dimographics[['user_id','gender','organized']].groupby(['user_id','gender','organized']).size()
age_organized = Data_features_dimographics[['user_id','age','organized']].groupby(['user_id','age','organized']).size()
gender_organized.to_frame().to_csv('gender_organized.csv')
age_organized.to_frame().to_csv('age_organized.csv')
gender_organized = pd.read_csv('gender_organized.csv')
gender_organized.head()

,user_id,gender,organized,0
0,1234567890,Female,Strongly_Agree,3
1,1278,Male,Disagree,3
2,Abc,Male,Agree,3
3,Abdo,Male,Agree,3
4,Abdo_mahdi,Male,Agree,3


In [269]:
gender_organized.groupby(['organized','gender']).size()

organized          gender        
Agree              Female            24
                   Male              74
                   refuse_to_disc     2
Disagree           Female             8
                   Male              16
                   refuse_to_disc     1
Neutral            Female            13
                   Male              43
Strongly_Agree     Female            12
                   Male              26
                   refuse_to_disc     1
Strongly_Disagree  Female             2
                   Male               3
                   refuse_to_disc     1
dtype: int64

In [270]:
age_organized.groupby(['organized','age']).size()

organized          age     
Agree              b18_24      40
                   b25_35      42
                   b35_45      11
                   b45_55       3
                   under_18     4
Disagree           b18_24       7
                   b25_35      11
                   b35_45       3
                   b45_55       1
                   under_18     3
Neutral            b18_24      30
                   b25_35      15
                   b35_45       4
                   b45_55       2
                   bover_55     1
                   under_18     4
Strongly_Agree     b18_24      17
                   b25_35      13
                   b35_45       4
                   b45_55       3
                   under_18     2
Strongly_Disagree  b18_24       2
                   b25_35       2
                   b35_45       1
                   bover_55     1
dtype: int64

In [275]:
organized_features = Data_features_dimographics.groupby(['organized','features']).size()
organized_features.to_frame().to_csv('organized_features.csv')
organized_features = pd.read_csv('organized_features.csv')
organized_features

,organized,features,0
0,Agree,Crime Rate,20
1,Agree,Cultural & Linguistic Similarities,18
2,Agree,Education Quality,50
3,Agree,Family Member Abroad,10
4,Agree,Health Care,44
5,Agree,Income Difference,37
6,Agree,Political Insecurity,26
7,Agree,Shorter Distance,6
8,Agree,Social Conflict,20
9,Agree,Wars & Dictatorship,19


#### enthusiastic/ features


In [276]:
t = Data_features_dimographics[['gender','age','enthusiastic','features']].groupby(['gender','age','enthusiastic','features']).size()
t.to_frame().to_csv('gender_enthusiastic_features.csv')
features_enthusiastic_data = pd.read_csv('gender_enthusiastic_features.csv')

In [277]:
features_enthusiastic_data.head()

,gender,age,enthusiastic,features,0
0,Female,b18_24,Agree,Education Quality,4
1,Female,b18_24,Agree,Family Member Abroad,2
2,Female,b18_24,Agree,Health Care,2
3,Female,b18_24,Agree,Income Difference,2
4,Female,b18_24,Agree,Political Insecurity,3


In [278]:
gender_enthusiastic = Data_features_dimographics[['user_id','gender','enthusiastic']].groupby(['user_id','gender','enthusiastic']).size()
age_enthusiastic = Data_features_dimographics[['user_id','age','enthusiastic']].groupby(['user_id','age','enthusiastic']).size()
gender_enthusiastic.to_frame().to_csv('gender_enthusiastic.csv')
age_enthusiastic.to_frame().to_csv('age_enthusiastic.csv')
gender_enthusiastic = pd.read_csv('gender_enthusiastic.csv')
gender_enthusiastic.head()

,user_id,gender,enthusiastic,0
0,1234567890,Female,Disagree,3
1,1278,Male,Disagree,3
2,Abc,Male,Agree,3
3,Abdo,Male,Agree,3
4,Abdo_mahdi,Male,Agree,3


In [280]:
gender_enthusiastic.groupby(['enthusiastic','gender']).size()

enthusiastic       gender        
Agree              Female            15
                   Male              48
                   refuse_to_disc     1
Disagree           Female            16
                   Male              37
Neutral            Female            20
                   Male              45
                   refuse_to_disc     1
Strongly_Agree     Female             6
                   Male              19
                   refuse_to_disc     1
Strongly_Disagree  Female             2
                   Male              13
                   refuse_to_disc     2
dtype: int64

In [281]:
age_enthusiastic.groupby(['enthusiastic','age']).size()

enthusiastic       age     
Agree              b18_24      25
                   b25_35      27
                   b35_45       6
                   b45_55       1
                   under_18     5
Disagree           b18_24      22
                   b25_35      18
                   b35_45       7
                   b45_55       4
                   bover_55     1
                   under_18     1
Neutral            b18_24      32
                   b25_35      23
                   b35_45       5
                   b45_55       2
                   under_18     4
Strongly_Agree     b18_24      12
                   b25_35       8
                   b35_45       3
                   b45_55       2
                   under_18     1
Strongly_Disagree  b18_24       5
                   b25_35       7
                   b35_45       2
                   bover_55     1
                   under_18     2
dtype: int64

In [282]:
enthusiastic_features = Data_features_dimographics.groupby(['enthusiastic','features']).size()
enthusiastic_features.to_frame().to_csv('enthusiastic_features.csv')
enthusiastic_features = pd.read_csv('enthusiastic_features.csv')
enthusiastic_features

,enthusiastic,features,0
0,Agree,Crime Rate,14
1,Agree,Cultural & Linguistic Similarities,10
2,Agree,Education Quality,36
3,Agree,Family Member Abroad,4
4,Agree,Health Care,30
5,Agree,Income Difference,26
6,Agree,Political Insecurity,18
7,Agree,Shorter Distance,4
8,Agree,Social Conflict,17
9,Agree,Wars & Dictatorship,6


#### Kind/ features

In [283]:
t = Data_features_dimographics[['gender','age','kind','features']].groupby(['gender','age','kind','features']).size()
t.to_frame().to_csv('gender_kind_features.csv')
features_kind_data = pd.read_csv('gender_kind_features.csv')

In [284]:
features_kind_data.head()

,gender,age,kind,features,0
0,Female,b18_24,Agree,Crime Rate,1
1,Female,b18_24,Agree,Cultural & Linguistic Similarities,1
2,Female,b18_24,Agree,Education Quality,9
3,Female,b18_24,Agree,Family Member Abroad,4
4,Female,b18_24,Agree,Health Care,3


In [285]:
gender_kind = Data_features_dimographics[['user_id','gender','kind']].groupby(['user_id','gender','kind']).size()
age_kind = Data_features_dimographics[['user_id','age','kind']].groupby(['user_id','age','kind']).size()
gender_kind.to_frame().to_csv('gender_kind.csv')
age_kind.to_frame().to_csv('age_kind.csv')
gender_kind = pd.read_csv('gender_kind.csv')
gender_kind.head()

,user_id,gender,kind,0
0,1234567890,Female,Agree,3
1,1278,Male,Agree,3
2,Abc,Male,Strongly_Disagree,3
3,Abdo,Male,Agree,3
4,Abdo_mahdi,Male,Agree,3


In [286]:
gender_kind.groupby(['kind','gender']).size()

kind               gender        
Agree              Female            26
                   Male              84
                   refuse_to_disc     1
Disagree           Female             4
                   Male              12
Neutral            Female             9
                   Male              25
Strongly_Agree     Female            20
                   Male              37
                   refuse_to_disc     2
Strongly_Disagree  Male               4
                   refuse_to_disc     2
dtype: int64

In [287]:
age_kind.groupby(['kind','age']).size()

kind               age     
Agree              b18_24      46
                   b25_35      40
                   b35_45      14
                   b45_55       6
                   under_18     5
Disagree           b18_24       4
                   b25_35       6
                   b35_45       3
                   b45_55       1
                   under_18     2
Neutral            b18_24      17
                   b25_35      12
                   b35_45       2
                   b45_55       1
                   bover_55     1
                   under_18     1
Strongly_Agree     b18_24      27
                   b25_35      22
                   b35_45       4
                   b45_55       1
                   under_18     5
Strongly_Disagree  b18_24       2
                   b25_35       3
                   bover_55     1
dtype: int64

In [288]:
kind_features = Data_features_dimographics.groupby(['kind','features']).size()
kind_features.to_frame().to_csv('kind_features.csv')
kind_features = pd.read_csv('kind_features.csv')
kind_features

,kind,features,0
0,Agree,Crime Rate,29
1,Agree,Cultural & Linguistic Similarities,23
2,Agree,Education Quality,52
3,Agree,Family Member Abroad,10
4,Agree,Health Care,47
5,Agree,Income Difference,40
6,Agree,Political Insecurity,26
7,Agree,Shorter Distance,8
8,Agree,Social Conflict,19
9,Agree,Wars & Dictatorship,20


#### calm/Features


In [289]:
t = Data_features_dimographics[['gender','age','calm','features']].groupby(['gender','age','calm','features']).size()
t.to_frame().to_csv('gender_calm_features.csv')
features_calm_data = pd.read_csv('gender_calm_features.csv')

In [290]:
features_calm_data.head()

,gender,age,calm,features,0
0,Female,b18_24,Agree,Cultural & Linguistic Similarities,3
1,Female,b18_24,Agree,Education Quality,7
2,Female,b18_24,Agree,Family Member Abroad,3
3,Female,b18_24,Agree,Health Care,3
4,Female,b18_24,Agree,Income Difference,1


In [291]:
gender_calm = Data_features_dimographics[['user_id','gender','calm']].groupby(['user_id','gender','calm']).size()
age_calm = Data_features_dimographics[['user_id','age','calm']].groupby(['user_id','age','calm']).size()
gender_calm.to_frame().to_csv('gender_calm.csv')
age_calm.to_frame().to_csv('age_calm.csv')
gender_calm = pd.read_csv('gender_calm.csv')
gender_calm.head()


,user_id,gender,calm,0
0,1234567890,Female,Agree,3
1,1278,Male,Neutral,3
2,Abc,Male,Strongly_Agree,3
3,Abdo,Male,Agree,3
4,Abdo_mahdi,Male,Strongly_Agree,3


In [292]:
gender_calm.groupby(['calm','gender']).size()

calm               gender        
Agree              Female            21
                   Male              60
                   refuse_to_disc     1
Disagree           Female            10
                   Male               9
                   refuse_to_disc     2
Neutral            Female            18
                   Male              39
Strongly_Agree     Female             8
                   Male              51
                   refuse_to_disc     1
Strongly_Disagree  Female             2
                   Male               3
                   refuse_to_disc     1
dtype: int64

In [295]:
age_calm.groupby(['calm','age']).size()

calm               age     
Agree              b18_24      45
                   b25_35      20
                   b35_45      12
                   b45_55       4
                   under_18     1
Disagree           b18_24       6
                   b25_35       8
                   b35_45       3
                   b45_55       3
                   under_18     1
Neutral            b18_24      25
                   b25_35      23
                   b35_45       4
                   b45_55       1
                   bover_55     1
                   under_18     3
Strongly_Agree     b18_24      20
                   b25_35      30
                   b35_45       2
                   b45_55       1
                   under_18     7
Strongly_Disagree  b25_35       2
                   b35_45       2
                   bover_55     1
                   under_18     1
dtype: int64

In [294]:
calm_features = Data_features_dimographics.groupby(['calm','features']).size()
calm_features.to_frame().to_csv('calm_features.csv')
calm_features = pd.read_csv('calm_features.csv')
calm_features

,calm,features,0
0,Agree,Crime Rate,25
1,Agree,Cultural & Linguistic Similarities,19
2,Agree,Education Quality,41
3,Agree,Family Member Abroad,8
4,Agree,Health Care,39
5,Agree,Income Difference,20
6,Agree,Political Insecurity,22
7,Agree,Shorter Distance,3
8,Agree,Social Conflict,17
9,Agree,Wars & Dictatorship,15
